In [1]:
import json

In [2]:
class Config:
    ratios_path = '/home/baskar/Desktop/StockPricePred/data/FMP'
    prices_path = '/home/baskar/Desktop/StockPricePred/data/NASDAQ_100'

config = Config()

In [ ]:
import json
import pandas as pd
from pathlib import Path
import numpy as np
import os

class RatiosProcessor:
    def __init__(self, ratios_path:str):
        self.ratios_path = Path(ratios_path)
    
    def load_ratios(self):
        all_data = []

        for filename in os.listdir(self.ratios_path):
            if filename.endswith('.json'):
                file_path = os.path.join(self.ratios_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        data = json.load(file)

                        if isinstance(data, dict):
                            all_data.append(data)
                        elif isinstance(data, list):
                            all_data.extend(data)
                        else:
                            print(f"Skipping {filename}: Unsupported JSON format")
                except Exception as e:
                    print(f"Error reading {filename}: {e}")

        return pd.DataFrame(all_data)
    
    def fill_missing_values(self):
        ratios_df = self.load_ratios()
        ratios_df['date'] = pd.to_datetime(ratios_df['date'])
        ratios_df = ratios_df.sort_values(by=['symbol', 'date']).reset_index(drop=True)

        numeric_cols = ratios_df.select_dtypes(include='number').columns
        ratios_df_filled = ratios_df.copy()

        for symbol in ratios_df['symbol'].unique():
            symbol_mask = ratios_df['symbol'] == symbol #note
            symbol_df = ratios_df[symbol_mask].sort_values(by='date').reset_index()

            for col in numeric_cols:
                for i in range(len(symbol_df)):
                    val = symbol_df.loc[i, col] #note
                    if pd.isna(val):
                        original_idx = symbol_df.loc[i, 'index']
                        prev_val = np.nan
                        next_val = np.nan

                        #look for previous value
                        for j in range(i-1, -1 -1):
                            prev_val = symbol_df.loc[j, col]
                            if not pd.isna(prev_val):
                                ratios_df_filled.at[original_ix, col] = prev_val
                                break
                            else:
                                for k in range(i+1, len(symbol_df)):
                                    next_val = symbol_df.loc[k, col]
                                    if not pd.isna(next_val):
                                        ratios_df_filled.at[original_idx, col] = next_val
                                        break
        return ratios_df_filled

    def get_missing_quarters(self):
        ratios_df = fill_missin_values()
        ratios_df['calenderYear'] = ratios_df['calenderYear'].astype(str)
        ratios_df['periods'] = ratios_df['periods'].str.upper()

        all_quarters = ['Q1', 'Q2', 'Q3', 'Q4']
        quarter_months = {'Q1' : '03-31', 'Q2' : '06-30', 'Q3' : '09-30', 'Q4' : '12-31'}

        rows_to_add = []

        grouped = ratios_df.groupby(['symbol', 'calenderYear'])

        for (symbol, year), group in grouped:
            present_quarters = all_quarters[:i]
                    
    


rp = RatiosProcessor(config.ratios_path)
rp.fill_missing_values()

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,AAPL,1985-09-30,1985,Q4,2.783006,2.217671,1.140826,48.371979,0.0,48.371979,...,2.377668,0.000000,0.000000,0.000000,0.000000,0.000000,2.377668,NaN,-0.468031,1.769538
1,AAPL,1985-12-31,1986,Q1,0.000000,0.000000,0.000000,NaN,0.0,NaN,...,2.545548,0.000000,0.000000,0.000000,0.000000,0.000000,2.545548,NaN,NaN,0.000000
2,AAPL,1986-03-31,1986,Q2,0.000000,0.000000,0.000000,NaN,0.0,NaN,...,4.328485,0.000000,0.000000,0.000000,0.000000,0.000000,4.328485,NaN,NaN,0.000000
3,AAPL,1986-06-30,1986,Q3,0.000000,0.000000,0.000000,NaN,0.0,NaN,...,5.094561,0.000000,0.000000,0.000000,0.000000,0.000000,5.094561,NaN,NaN,0.000000
4,AAPL,1986-09-30,1986,Q4,3.168645,2.837747,1.754033,46.356695,0.0,46.356695,...,4.107259,0.000000,0.000000,0.000000,0.000000,0.000000,4.107259,NaN,-1.361657,3.022602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10888,ZS,2024-01-31,2024,Q2,1.904860,1.904860,0.908253,82.219185,0.0,82.219185,...,66.863522,-308.258827,348.288310,247.086149,247.086149,17.724883,66.863522,0.0,19323.580382,36.609545
10889,ZS,2024-04-30,2024,Q3,1.666305,1.666305,0.740129,82.367096,0.0,82.367096,...,46.983199,339.771395,211.093941,149.879206,149.879206,-2.017393,46.983199,0.0,586.131788,23.776622
10890,ZS,2024-07-31,2024,Q4,1.091147,1.091147,0.457158,111.808379,0.0,111.808379,...,45.251640,-450.804024,196.806358,131.797232,131.797232,2.553574,45.251640,0.0,1156.988627,21.056594
10891,ZS,2024-10-31,2025,Q1,1.145151,1.145151,0.524241,60.850809,0.0,60.850809,...,43.921587,-572.167871,83.241372,83.241372,83.241372,27.771075,43.921587,0.0,1117.762874,19.306903
